# Extracting Actor Etnicity 

In this notebook we extract the actors etnicty from the actros photos and match them to movie posters.

Install of runing on colab:

In [ ]:
!pip install cython

!pip install insightface
!pip install -U retinaface_pytorch
!pip install onnxruntime

!pip install opencv-python-headless==4.1.2.30
!pip install imagehash
!pip install IMDbPY

In [1]:
%load_ext autoreload
%autoreload 2
%aimport
%matplotlib inline
import pickle
from pathlib import Path

import numpy as np
from scipy.spatial.distance import cdist

from tqdm import tqdm
from io import BytesIO

import pandas as pd
import requests
from PIL import Image

import cv2
import matplotlib.pyplot as plt

from collections import Counter
import imdb

# Retina Face Model
from retinaface.pre_trained_models import get_model

# Insight Face Model
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
from insightface.utils import face_align
from insightface.model_zoo import ArcFaceONNX

Modules to reload:
all-except-skipped

Modules to skip:



ModuleNotFoundError: No module named 'retinaface'

In [2]:
posters_folder = Path('posters')
movies_folder = Path('movies')
posters_folder.mkdir(exist_ok=True)
movies_folder.mkdir(exist_ok=True)

# Face Recognition
Here we recieve the dataframe after the classification of duplicate posters

In [ ]:
!unzip actors.zip # the sample'sn actors folder with images 
!unzip posters.zip # the sample's posters images 
!unzip movies.zip # the sample's movies metadata 

In [ ]:
# the base df is posters_with_dup

posters_with_dup = pd.read_pickle("posters_with_dup.pkl")
posters_df = posters_with_dup[posters_with_dup["dups"] == False]

## Posters

### Detection
Here we detect every face in each poster using RetinFace model.

In [ ]:
model = get_model("resnet50_2020-07-20", max_size=2048)
model.eval()

Downloading: "https://github.com/ternaus/retinaface/releases/download/0.01/retinaface_resnet50_2020-07-20-f168fae3c.zip" to /root/.cache/torch/hub/checkpoints/retinaface_resnet50_2020-07-20-f168fae3c.zip


  0%|          | 0.00/96.9M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/hub.py:480: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


In [ ]:
from tqdm import tqdm
import traceback

posters_df['faces'] = ''

try:
    for row_num, (i, poster_df) in enumerate(tqdm(posters_df.iterrows(), total=len(posters_df))):
        image_path = posters_folder / f'{poster_df["tconst"]}{poster_df["file_path"]}'
        image = cv2.cvtColor(cv2.imread(str(image_path)), cv2.COLOR_BGR2RGB)
        posters_df.at[i,'faces'] = model.predict_jsons(image)
        if row_num % 1000 == 0:
            posters_df.to_pickle("posters_face_encodings.pkl") 
            print('passed', row_num)
except Exception as e:
    print(traceback.format_exc())
    
posters_df.to_pickle("posters_face_encodings.pkl")

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  2%|▏         | 1/53 [00:17<15:13, 17.57s/it]

passed 0


100%|██████████| 53/53 [14:52<00:00, 16.84s/it]


### Encoding
Here we encode each face detected by RetinaFace using the ArcFace encoding.

In [ ]:
posters_face_encodings = pd.read_pickle("posters_face_encodings.pkl")

In [ ]:
class ArcFace(ArcFaceONNX):
    def get(self, img, landmark):
        aimg = face_align.norm_crop(img, landmark=landmark)
        return self.get_feat(aimg).flatten()


arc = ArcFace("w600k_r50.onnx")

In [ ]:
import traceback
from tqdm import tqdm_notebook as tqdm

posters_face_encodings["embedding"] = ""

for row_num, (i, poster_df) in enumerate(tqdm(posters_face_encodings.iterrows(), total=len(posters_face_encodings))):
    image_path = posters_folder / f'{poster_df["tconst"]}{poster_df["file_path"]}'

    try:
        if poster_df["embedding"] == "":
            img = cv2.imread(str(image_path))
            posters_face_encodings.at[i, "embedding"] = [
                arc.get(img, np.array(f["landmarks"]))
                for f in poster_df["faces"]
                if f["score"] > 0
            ]
            if row_num + 1 % 10000 == 0:
                posters_face_encodings.to_pickle("posters_face_encodings.pkl")
    except Exception as e:
        print(traceback.format_exc())

posters_face_encodings.to_pickle("posters_face_encodings.pkl")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/53 [00:00<?, ?it/s]

## Actors
Here we are doing the same process of detection and embeding but for the actors headshots.

In [ ]:
actors_df = pd.read_pickle('actors_df.pkl')

In [ ]:
app = FaceAnalysis(allowed_modules=["detection", "recognition"])
app.prepare(ctx_id=0, det_size=(640, 640))

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [04:34<00:00, 1026.83KB/s]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [ ]:
# Actors Insight Face

import traceback
from tqdm import tqdm

actors_df["faces"] = ""

try:
    for row_num, (i, actor_df) in enumerate(tqdm(actors_df.iterrows(), total=len(actors_df))):
        image_path = actor_df["img_path"]
        img = cv2.imread(str(image_path))
        actors_df.at[i, "faces"] = [dict(f) for f in app.get(img)]

        if row_num % 10000 == 0:
            actors_df.to_pickle("actors_face_encodings.pkl")
            print("passed", row_num)

except Exception as e:
    print(traceback.format_exc())

actors_df.to_pickle("actors_face_encodings.pkl")

  0%|          | 1/359 [00:00<03:56,  1.51it/s]

passed 0


100%|██████████| 359/359 [03:23<00:00,  1.77it/s]


## Match Posters Faces With Actors Ids
Here we match each face found in the poster to an actor from the movies cast list using a threshold

In [ ]:
posters_face_encodings = pd.read_pickle('posters_face_encodings.pkl')
actors_face_encodings = pd.read_pickle('actors_face_encodings.pkl')

In [ ]:
posters_face_encodings  = posters_face_encodings.rename(columns={"embedding":"face_encoding"})
posters_face_encodings["boxes"] = posters_face_encodings["faces"].apply(lambda faces: [f["bbox"] for f in faces])

actors_face_encodings["face_encoding"] = actors_face_encodings["faces"].apply(lambda faces: [f["embedding"] for f in faces])
actors_face_encodings["boxes"] = actors_face_encodings["faces"].apply(lambda faces: [f["bbox"] for f in faces])

In [ ]:
# box = (left, top, right, bottom)
MAX_CAST_ACTORS = 1000 # 1000 to get all the cast
TOLERANCE = 0.7

actors_face_encodings["face_size"] = actors_face_encodings["boxes"].apply(lambda x: [(b - t) * (r - l) for l, t, r, b in x])
actors_face_encodings = actors_face_encodings[actors_face_encodings["faces"].apply(lambda x: len(x) != 0)]
actors_face_encodings["max_face"] = actors_face_encodings.apply(lambda x: x["face_encoding"][np.argmax(x["face_size"])], axis=1)

posters_df = posters_face_encodings[posters_face_encodings["face_encoding"].apply(lambda x: len(x) != 0)]
movies = set(posters_df["movie"])

posters_df["face_actor"] = ""
posters_df["face_distance"] = ""
posters_df["face_distance"] = posters_df["face_distance"].astype(object)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [ ]:
actors_face_encodings.to_pickle("actors_face_encodings.pkl")

In [ ]:
for movie, posters in tqdm(posters_df.groupby("movie"), total=len(movies)):
    try:
        with (movies_folder / f'{movie}-{posters["tconst"].iloc[0].lstrip("t")}.pkl').open("rb") as f:
            movie_info = pickle.load(f)
    except (KeyError, FileNotFoundError):
        try:
            with next(movies_folder.glob(f'*{posters["tconst"].iloc[0].lstrip("t")}.pkl')).open("rb") as f:
                movie_info = pickle.load(f)
        except StopIteration:
            continue

    try:
        cast = movie_info["cast"][:MAX_CAST_ACTORS]
        cast_ids = [a.getID() for a in cast]

        actors = actors_face_encodings[actors_face_encodings["imdb_id"].isin(cast_ids)]

        if len(actors):
            for i, p in posters.iterrows():
                dist = cdist(np.stack(p["face_encoding"]), np.stack(actors["max_face"].values), metric='cosine')
                mins = np.min(dist, 1)

                posters_df.loc[i, "face_distance"] = [[mins]]
                actors_ids = actors.iloc[np.argmin(dist, 1)]["imdb_id"].values

                matched_actors_ids = [[np.asarray([a[0] if a[1] < TOLERANCE else "" for a in zip(actors_ids, mins)],"object",)]]

                posters_df.loc[i, "face_actor"] = matched_actors_ids
    except KeyError:
        pass



  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)
100%|██████████| 3/3 [00:00<00:00,  7.78it/s]


In [ ]:
posters_df.to_pickle("match_poster_actor_cast_all.pkl")

# Ethnicity
Here we use FairFace model to recognize the actors ethnicity based on 3 headshots

## Predict Actors Races

In [ ]:
actors_face_encodings = pd.read_pickle('actors_face_encodings.pkl')

In [ ]:
# Clone from git and download 4 race model from drive

!git clone https://github.com/dchen236/FairFace.git

Cloning into 'FairFace'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 220 (delta 4), reused 1 (delta 0), pack-reused 211
Receiving objects: 100% (220/220), 14.23 MiB | 20.07 MiB/s, done.
Resolving deltas: 100% (111/111), done.
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1kXdAsqT8YiNYIMm8p5vQUvNFwhBbT4vQ 



In [ ]:
# download the fairface 4-race model
!gdown --id 1kXdAsqT8YiNYIMm8p5vQUvNFwhBbT4vQ

# download the fairface 7-race model
# !gdown --id 113QMzQzkBDmYMs9LwzvD-jxEZdBQ5J4X

In [ ]:
import dlib
import os

def get_face(image_path, kps, default_max_size=800,size = 300, padding = 0.25):
    
    img = dlib.load_rgb_image(str(image_path))

    return face_align.norm_crop(img, kps, size)

In [ ]:
# We adjusted FairFace predict function to exclude 7 races model and image cropping

import torch, torchvision 
import torch.nn as nn
from torchvision import datasets, models, transforms
from tqdm import tqdm

def predidct_race(save_prediction_at, actors_df):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    model_fair_4 = torchvision.models.resnet34(pretrained=True)
    model_fair_4.fc = nn.Linear(model_fair_4.fc.in_features, 18)
    model_fair_4.load_state_dict(torch.load('res34_fair_align_multi_4_20190809.pt', map_location=torch.device(device)))
    model_fair_4 = model_fair_4.to(device)
    model_fair_4.eval()

    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    face_names = []
    race_scores_fair_4 = []
    race_preds_fair_4 = []

    for index, (_, actor_img) in enumerate(tqdm(actors_df.iterrows(), total=len(actors_df))):

        img_path = actor_img["img_path"]
        face_names.append(img_path)

        kps = actor_img["faces"][0]['kps']  
        image = get_face(img_path, kps)
        image = trans(image)
        image = image.view(1, 3, 224, 224)  # reshape image to match model dimensions (1 batch size)
        image = image.to(device) 
        
        # fair 4 class
        outputs = model_fair_4(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        race_outputs = outputs[:4]
        race_score = np.exp(race_outputs) / np.sum(np.exp(race_outputs))
        race_pred = np.argmax(race_score)

        race_scores_fair_4.append(race_score)
        race_preds_fair_4.append(race_pred)


    result = pd.DataFrame([face_names,
                           race_preds_fair_4,
                           race_scores_fair_4,
                         ]).T
    result.columns = ['face_name_align',
                      # 'race_preds_fair',
                      'race_preds_fair_4',
                      # 'race_scores_fair',
                      'race_scores_fair_4']

    # race fair 4

    result.loc[result['race_preds_fair_4'] == 0, 'race4'] = 'White'
    result.loc[result['race_preds_fair_4'] == 1, 'race4'] = 'Black'
    result.loc[result['race_preds_fair_4'] == 2, 'race4'] = 'Asian'
    result.loc[result['race_preds_fair_4'] == 3, 'race4'] = 'Indian'

    result[['face_name_align',
            'race4',
            'race_scores_fair_4']].to_csv(save_prediction_at, index=False)

    print("saved results at ", save_prediction_at)


In [ ]:
predidct_race('races.csv', actors_face_encodings)

In [ ]:
# Here we recognize and later fillter black&white headshots

from PIL import Image, ImageStat
def is_grayscale(file, thumb_size=40, MSE_cutoff=22, adjust_color_bias=True):
    pil_img = Image.open(file)
    bands = pil_img.getbands()
    if bands == ('R','G','B') or bands== ('R','G','B','A'):
        thumb = pil_img.resize((thumb_size,thumb_size))
        SSE, bias = 0, [0,0,0]
        if adjust_color_bias:
            bias = ImageStat.Stat(thumb).mean[:3]
            bias = [b - sum(bias)/3 for b in bias ]
        for pixel in thumb.getdata():
            mu = sum(pixel)/3
            SSE += sum((pixel[i] - mu - bias[i])*(pixel[i] - mu - bias[i]) for i in [0,1,2])
        MSE = float(SSE)/(thumb_size*thumb_size)
        if MSE <= MSE_cutoff:
            return True, MSE
        else:
            return False, MSE
    elif len(bands)==1:
        return True, 0
    else:
        return False, 0

In [ ]:
races = pd.read_csv('races.csv')

In [ ]:
for i, line in tqdm(races.iterrows(), total=len(races)):
    is_grey, mse = is_grayscale(line["face_name_align"])
    races.loc[i,"is_grey"] = is_grey
    races.loc[i,"mse"] = mse

 54%|█████▍    | 192/356 [00:11<00:10, 14.97it/s]/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (95727189 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
100%|██████████| 356/356 [00:25<00:00, 14.00it/s]


In [ ]:
races.to_csv('races.csv', index=False)

## Connect Faces on Poster to Etnicity

Here we clean the data after prediction and combine it to a large dataframe

In [ ]:
# clean ethnicity data

fair_face_outputs = pd.read_csv('races.csv')
fair_face_outputs = fair_face_outputs[fair_face_outputs['mse'] > 30]
fair_face_outputs = fair_face_outputs[~fair_face_outputs["race4"].isna()]
fair_face_outputs["id"] = fair_face_outputs["face_name_align"].apply(lambda x: x.partition("actors/")[2].partition('/')[0])
fair_face_outputs["imdb_id"] = fair_face_outputs['id'].str.rpartition("-")[2]
fair_face_outputs["name"] = fair_face_outputs['id'].str.rpartition("-")[0]

fair_face_outputs["race_scores_fair_4"] = fair_face_outputs["race_scores_fair_4"].str.strip("[")
fair_face_outputs["race_scores_fair_4"] = fair_face_outputs["race_scores_fair_4"].str.strip("]")
fair_face_outputs["race_scores_fair_4"] = fair_face_outputs["race_scores_fair_4"].str.strip(" ")

fair_face_outputs["race_scores_fair_4"] = fair_face_outputs["race_scores_fair_4"].str.replace("\n", "")
fair_face_outputs["race_scores_fair_4"] = fair_face_outputs["race_scores_fair_4"].str.replace("  ", " ")
fair_face_outputs["race_scores_fair_4"] = fair_face_outputs["race_scores_fair_4"].str.replace("  ", " ")
fair_face_outputs["race_scores_fair_4"] = fair_face_outputs["race_scores_fair_4"].str.replace("  ", " ")

fair_face_outputs[["White", "Black", "Asian", "Indian"]] = pd.DataFrame(
    fair_face_outputs["race_scores_fair_4"].str.split(" ", expand=True),
    index=fair_face_outputs.index,
)

fair_face_outputs["White"] = fair_face_outputs["White"].astype(float)
fair_face_outputs["Black"] = fair_face_outputs["Black"].astype(float)
fair_face_outputs["Asian"] = fair_face_outputs["Asian"].astype(float)
fair_face_outputs["Indian"] = fair_face_outputs["Indian"].astype(float)

In [ ]:
ethni_df = fair_face_outputs.groupby("imdb_id").mean().reset_index()
ethni_df["race"] = ethni_df[["White", "Black", "Asian", "Indian"]].idxmax(axis=1)
ethni_df = ethni_df[["imdb_id", "race"]]
ethni_df.index = ethni_df.imdb_id

In [ ]:
match_poster_actor = pd.read_pickle("match_poster_actor_cast_all.pkl")

match_poster_actor = match_poster_actor[match_poster_actor['face_actor'].apply(lambda x: len(x)>0)]
match_poster_actor['face_actor'] = match_poster_actor['face_actor'].apply(lambda x: x[0])
match_poster_actor = match_poster_actor[match_poster_actor['face_actor'].apply(lambda x: any(x))]

In [ ]:
from collections import Counter

def align_faces_with_ethnicity(row):
  return len(row["face_actor"]) == len(row["faces"])

match_poster_actor = match_poster_actor[match_poster_actor.apply(align_faces_with_ethnicity, axis=1)]

def find_faces_races(row):
  races=[]
  for imdb_id in row['face_actor']:
    try:
      values = ethni_df.loc[imdb_id]['race']
      races.append(values)
    except KeyError:
      races.append('')

  row['faces_races'] = races
  return row

posters_races_df = match_poster_actor.apply(find_faces_races, axis=1)


def get_races_counts(row):
  races=row["faces_races"]

  keys = Counter(races).keys()
  counts = [100*count/len(races) for count in Counter(races).values()]

  for key, count in zip(keys,counts):
    row[key] = count

  return row

posters_races_df = posters_races_df.apply(get_races_counts, axis = 1).fillna(0).drop([''], axis=1)

tqdm.pandas()

# images size was reduced during encoding
def get_poster_image(movie_title, poster_path):
    return Image.open(posters_folder / f"{movie_title}{poster_path}")
    
def calc_new_img_size(row):
    poster_img = get_poster_image(row["tconst"], row["file_path"])
    w,h = poster_img.size
    row['width'] = w
    row['height'] = h
    
    return row

posters_races_df = posters_races_df.progress_apply(calc_new_img_size, axis=1)


def calc_face_percentage(row):
    result = []
    for box in row["boxes"]:
        l, t, r, b = box
        face_size = (b - t) * (r - l)
        poster_size = row["height"]*row['width']
        result.append( face_size / poster_size )

    return result

posters_races_df["face_percentage"] = posters_races_df.apply(calc_face_percentage, axis=1)

posters_races_df["face_percentage_relative"] = posters_races_df["face_percentage"].apply(lambda row: np.array(row) / max(row))

posters_races_df["faces_count"] = posters_races_df["faces"].apply(lambda x: len(x))

100%|██████████| 21/21 [00:00<00:00, 849.90it/s]


In [ ]:
posters_races_df.to_pickle("posters_new_races4_cast_all.pkl")

## Create Ranking Dataframe

Here we create another dataframe that contains the actor position in the cast list - "Rank". 

We do it for each actor we recognized in the posters

In [ ]:
def get_actors_ranking(posters):
    movie = posters['movie']
    tconst = posters['tconst']
    actor_id = posters['face_actor']

        
    try:
        movie_info = pd.read_pickle(movies_folder / f'{movie}-{tconst.lstrip("t")}.pkl')
    except (KeyError, FileNotFoundError):
        try:
            with next(movies_folder.glob(f'*{tconst.lstrip("t")}.pkl')).open("rb") as f:
                movie_info = pickle.load(f)
        except StopIteration:
            print(f'{movie}-{tconst}')
            return []
                
    cast = movie_info['cast']
    cast_ids = [a.getID() for a in cast]

    posters['actor_rank'] = cast_ids.index(actor_id)+1
        
    return posters

from tqdm import tqdm
tqdm.pandas()


posters_for_ranking_graphs = posters_races_df[['movie', 'startYear', 'tconst', 'face_actor', 'file_path', 'faces_races']].explode('face_actor')
posters_for_ranking_graphs['face_race'] = posters_races_df[['movie', 'startYear', 'tconst', 'face_actor', 'file_path', 'faces_races']].explode('faces_races')['faces_races']
posters_for_ranking_graphs = posters_for_ranking_graphs[['movie', 'startYear', 'tconst', 'face_actor', 'file_path', 'face_race']]
posters_for_ranking_graphs = posters_for_ranking_graphs[posters_for_ranking_graphs['face_actor'] != '']
posters_for_ranking_graphs = posters_for_ranking_graphs[posters_for_ranking_graphs['face_race'] != '']
posters_for_ranking_graphs = posters_for_ranking_graphs.progress_apply(get_actors_ranking, axis=1)

100%|██████████| 50/50 [00:00<00:00, 72.21it/s]


In [ ]:
posters_for_ranking_graphs.to_pickle("ranking_posters_new_races4_cast_all.pkl")